# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/fabian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fabian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM-SAT - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'e5abeb'. Skipping!
Property 'summary' already exists in node 'b92034'. Skipping!
Property 'summary' already exists in node 'f5a20d'. Skipping!
Property 'summary' already exists in node '803efe'. Skipping!
Property 'summary' already exists in node '7b0d6b'. Skipping!
Property 'summary' already exists in node 'f89890'. Skipping!
Property 'summary' already exists in node '427389'. Skipping!
Property 'summary' already exists in node '5e37ee'. Skipping!
Property 'summary' already exists in node '9a20d8'. Skipping!
Property 'summary' already exists in node '2861ad'. Skipping!
Property 'summary' already exists in node '9bce39'. Skipping!
Property 'summary' already exists in node 'b3e971'. Skipping!
Property 'summary' already exists in node 'dfd8a0'. Skipping!
Property 'summary' already exists in node '5b3b37'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'e5abeb'. Skipping!
Property 'summary_embedding' already exists in node '7b0d6b'. Skipping!
Property 'summary_embedding' already exists in node 'f89890'. Skipping!
Property 'summary_embedding' already exists in node 'b3e971'. Skipping!
Property 'summary_embedding' already exists in node '427389'. Skipping!
Property 'summary_embedding' already exists in node 'f5a20d'. Skipping!
Property 'summary_embedding' already exists in node 'b92034'. Skipping!
Property 'summary_embedding' already exists in node '9a20d8'. Skipping!
Property 'summary_embedding' already exists in node '5b3b37'. Skipping!
Property 'summary_embedding' already exists in node '2861ad'. Skipping!
Property 'summary_embedding' already exists in node '9bce39'. Skipping!
Property 'summary_embedding' already exists in node '5e37ee'. Skipping!
Property 'summary_embedding' already exists in node 'dfd8a0'. Skipping!
Property 'summary_embedding' already exists in node '803efe'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:
1. SingleHopSpecificQuerySynthesizer - It picks a single node/chunk in the graph (e.g. a paragraph that contains a named entity) and asks a direct, factual question that can be answered from that snippet alone.
2. MultiHopSpecificQuerySynthesizer - It selects multiple graph nodes that share overlapping entities, weaves a key-phrase across them, and forms a question whose answer must pull those pieces together.
3. MultiHopAbstractQuerySynthesizer - It first summarises a cluster of semantically related nodes into a theme and then generates a higher-level, conceptual question about that theme.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of Volume 2 in the c...,"[Chapter 1 Academic Years, Academic Calendars,...",The context provided does not specify the sign...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) specify?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) specifies the weeks of instruc...,single_hop_specifc_query_synthesizer
2,What is Chapter 3 in the context of educationa...,[Inclusion of Clinical Work in a Standard Term...,"In the provided context, Volume 8, Chapter 3 o...",single_hop_specifc_query_synthesizer
3,Could you explain how the Federal Work-Study p...,[Non-Term Characteristics A program that measu...,The Federal Work-Study (FWS) program is an exc...,single_hop_specifc_query_synthesizer
4,Volume 8?,[both the credit or clock hours and the weeks ...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
5,How does credit hour allocation for clinical e...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work includ...,multi_hop_abstract_query_synthesizer
6,how credit-hour and clock-hour programs follow...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that for credit-hour prog...,multi_hop_abstract_query_synthesizer
7,Considering the guidance and principles for de...,[<1-hop>\n\nboth the credit or clock hours and...,The proration of loan limits affects students ...,multi_hop_abstract_query_synthesizer
8,Volume 8 and Volume 7 which one is better for ...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that Volume 8 covers pror...,multi_hop_specific_query_synthesizer
9,How do Appendices A and B relate to the disbur...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A provides guidance on calculating Pe...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [35]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '91e79f'. Skipping!
Property 'summary' already exists in node '7c0895'. Skipping!
Property 'summary' already exists in node '31b084'. Skipping!
Property 'summary' already exists in node '1c80fe'. Skipping!
Property 'summary' already exists in node 'f5d0b3'. Skipping!
Property 'summary' already exists in node '138730'. Skipping!
Property 'summary' already exists in node '9ceab8'. Skipping!
Property 'summary' already exists in node 'd3406f'. Skipping!
Property 'summary' already exists in node 'fcdc93'. Skipping!
Property 'summary' already exists in node '8a05bd'. Skipping!
Property 'summary' already exists in node 'e14034'. Skipping!
Property 'summary' already exists in node '4c10ca'. Skipping!
Property 'summary' already exists in node '7b01b2'. Skipping!
Property 'summary' already exists in node '21d0c6'. Skipping!


...

Property 'summary_embedding' already exists in node '91e79f'. Skipping!
Property 'summary_embedding' already exists in node '1c80fe'. Skipping!
Property 'summary_embedding' already exists in node '31b084'. Skipping!
Property 'summary_embedding' already exists in node '138730'. Skipping!
Property 'summary_embedding' already exists in node 'e14034'. Skipping!
Property 'summary_embedding' already exists in node 'f5d0b3'. Skipping!
Property 'summary_embedding' already exists in node '9ceab8'. Skipping!
Property 'summary_embedding' already exists in node 'd3406f'. Skipping!
Property 'summary_embedding' already exists in node 'fcdc93'. Skipping!
Property 'summary_embedding' already exists in node '4c10ca'. Skipping!
Property 'summary_embedding' already exists in node '8a05bd'. Skipping!
Property 'summary_embedding' already exists in node '7c0895'. Skipping!
Property 'summary_embedding' already exists in node '7b01b2'. Skipping!
Property 'summary_embedding' already exists in node '21d0c6'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [36]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Could you please explain the role and signific...,"[Chapter 1 Academic Years, Academic Calendars,...",The context provided does not explicitly defin...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(a) specify regarding th...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) specifies the minimum number o...,single_hop_specifc_query_synthesizer
2,What is the significance of Volume 8 in the co...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What is the significance of Title IV in relati...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,How do scheduled payment periods and installme...,[<1-hop>\n\nboth the credit or clock hours and...,Scheduled payment periods determine the timing...,multi_hop_abstract_query_synthesizer
5,How does completing additional hours or weeks ...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in clock-hour or non...,multi_hop_abstract_query_synthesizer
6,How does the inclusion of clinical work in sta...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
7,Hwo do academic calenders comply with 34 CFR 6...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Academic calenders must define an academic yea...,multi_hop_abstract_query_synthesizer
8,Chapter 2 and Chapter 3 how do they relate in ...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Chapter 2 discusses the minimum weeks of instr...,multi_hop_specific_query_synthesizer
9,Chapter 2 and 3 what about rules?,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Chapter 2 talks about academic years and weeks...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [30]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data 2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [37]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [38]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [42]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [43]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [44]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [46]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [47]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct Subsidized Loans (available only to undergraduate students)\n- Direct Unsubsidized Loans\n- Direct PLUS Loans (including student Federal PLUS Loans and parent PLUS Loans on behalf of dependent students)\n- Subsidized and Unsubsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010)\n- Federal SLS Loans (also made under the FFEL Program before July 1, 2010)\n\nNote: No new FFEL Program loans have been made since July 1, 2010.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [48]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [49]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:
- `qa_evaluator`: Evaluates the factual correctness of the model's answer by comparing it to the reference answer, focusing on whether the response accurately addresses the user's question using the provided context.
- `labeled_helpfulness_evaluator`: Assesses how helpful the model's response is to the user, specifically considering if the answer aids the user in light of the correct reference answer.
- `empathy_evaluator`: Evaluates the degree of empathy in the model's response, determining whether the answer makes the user feel heard and understood.


## LangSmith Evaluation

In [50]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'shiny-opinion-64' at:
https://smith.langchain.com/o/bbbdc2e8-da22-470d-aa79-cd31d5ea3160/datasets/b05d07a9-bc3e-44e8-b5cc-125832cebc74/compare?selectedSessions=d9a9e5f6-274e-4b0c-afa0-a1209c20ce7f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does Volume 8 explain the impact of accele...,Based on the provided context from Volume 8:\n...,None,Volume 8 discusses that accelerated progressio...,0,0,0,5.415995,fc9eabc0-c48c-43fd-9487-8a91ac80c7ba,059a2a5f-0900-4187-a8c4-b68c579fc2b9
1,How do Volume 7 and Volume 8 relate to the tim...,Based on the provided context:\n\n- Volume 8 a...,None,Volume 8 discusses how the inclusion of clinic...,1,1,0,6.142738,018d89c8-a5c1-46d3-8eb3-74a42e8c43a3,fd15d821-b59d-466f-8b20-b69be9fa68af
2,Chapter 2 and 3 what about rules?,Based on the provided context:\n\n- Chapter 2 ...,None,Chapter 2 talks about academic years and weeks...,0,0,0,5.726322,1e87bd27-4470-4542-84c6-6b28a53f3182,4d3bb0a8-86fc-40ac-9e4e-f0b42a9f89b8
3,Chapter 2 and Chapter 3 how do they relate in ...,Chapter 2 discusses the Cost of Attendance (CO...,None,Chapter 2 discusses the minimum weeks of instr...,0,0,0,3.134898,357856f9-6e21-4f0b-893a-43f18d817161,90ffb704-c0be-44b2-841d-09e5242319ee
4,Hwo do academic calenders comply with 34 CFR 6...,"Based on the provided context, there is no spe...",None,Academic calenders must define an academic yea...,0,0,0,4.177557,c719285b-5354-4696-b502-cf41c9e94cd9,08db9457-476d-46f6-9597-3b184dc371ce
5,How does the inclusion of clinical work in sta...,The inclusion of clinical work in standard ter...,None,The inclusion of clinical work in standard ter...,1,1,0,4.408087,c9a07bf5-0785-44fa-b407-5674f07667c4,d87fa66c-a65e-4fbd-ac4d-22269badaa83
6,How does completing additional hours or weeks ...,Completing additional hours or weeks of instru...,None,The context explains that in clock-hour or non...,1,1,0,9.968549,0190f70a-d6b4-453f-8b4d-919ee56d08c3,5f070382-f09d-49b7-bab8-c740d8ca66d4
7,How do scheduled payment periods and installme...,"Based on the provided context, scheduled payme...",None,Scheduled payment periods determine the timing...,1,1,0,6.442671,dcec7e14-e485-42f8-965a-6847a23b4b36,1fb90465-b4de-4e6c-9fe4-7f3d053739bd
8,What is the significance of Title IV in relati...,Title IV programs (except Federal Work-Study) ...,None,The payment period is applicable to all Title ...,1,1,0,2.475866,3776eeb5-9d49-43d0-803e-d835b10eb4c0,f29c36cf-f775-43f0-8560-1e66dee45302
9,What is the significance of Volume 8 in the co...,Volume 8 provides additional guidance on certa...,None,Inclusion of clinical work in a standard term ...,1,1,0,2.715698,5805d52c-401c-4096-bf4f-4a26c242022a,4c460686-e16b-45e6-a318-a34f506bf3e7


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [61]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [62]:
rag_documents = docs

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:
Modifying the chunk size changes how much context is available to the model when answering questions. If the chunk size is too small, important information may be split across multiple chunks, making it harder for the retriever to find all relevant context and potentially leading to incomplete or less accurate answers. If the chunk size is too large, each chunk may contain unrelated or excessive information, which can introduce noise and make retrieval less precise. Larger chunks can also increase computational costs and may exceed model input limits. Therefore, tuning the chunk size helps balance context completeness and retrieval precision, directly impacting the application's performance. 


In [64]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:
Modifying the embedding model changes how your application represents and understands the meaning of text. Different embedding models vary in their ability to capture semantic relationships, context, and domain-specific nuances. A more advanced or better-suited embedding model can produce vector representations that more accurately reflect the true meaning and relationships in your data, leading to improved retrieval of relevant documents and more accurate answers. Conversely, a less capable or mismatched embedding model may result in poor retrieval performance, as it might not capture important distinctions or similarities in your data. Therefore, choosing and tuning the embedding model is crucial for optimizing the performance and accuracy of your RAG application.


In [65]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [66]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [67]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [68]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question; I understand that navigating loan options can feel overwhelming, and I'm here to help clarify things gently.\n\nBased on the context provided, there are several types of loans available to students and their parents:\n\n1. **Direct Subsidized Loans**: These loans are offered to students based on financial need, and the government pays the interest while the student is in school.\n\n2. **Direct Unsubsidized Loans**: These loans are available to students regardless of financial need, but interest accrues during all periods.\n\n3. **Direct PLUS Loans**: These are loans that parents of dependent students can take out to help cover the student's Cost of Attendance (COA), provided the parent meets eligibility requirements. Graduate or professional students may also apply for Direct PLUS Loans for themselves.\n\nIt's important to note that:\n\n- Parents of dependent students who cannot obtain a Direct PLUS Loan may allow the student to borrow additional Direct Un

Finally, we can evaluate the new chain on the same test set!

In [60]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'cold-education-13' at:
https://smith.langchain.com/o/bbbdc2e8-da22-470d-aa79-cd31d5ea3160/datasets/b05d07a9-bc3e-44e8-b5cc-125832cebc74/compare?selectedSessions=6fec04b6-f5c2-437d-820d-37bb9d441a73




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does Volume 8 explain the impact of accele...,Thank you for your thoughtful question. Based ...,None,Volume 8 discusses that accelerated progressio...,0,0,1,5.160549,fc9eabc0-c48c-43fd-9487-8a91ac80c7ba,65a8569e-ba43-4bf2-8326-f5ecf1b9b373
1,How do Volume 7 and Volume 8 relate to the tim...,Thank you for your thoughtful question about h...,None,Volume 8 discusses how the inclusion of clinic...,0,0,1,6.076393,018d89c8-a5c1-46d3-8eb3-74a42e8c43a3,77a57bf9-64a2-4005-ad2b-69df36e5f3bd
2,Chapter 2 and 3 what about rules?,Thank you for your question. Based on the cont...,None,Chapter 2 talks about academic years and weeks...,0,0,1,2.559147,1e87bd27-4470-4542-84c6-6b28a53f3182,12f9d501-f776-40a9-9597-7c5260698c51
3,Chapter 2 and Chapter 3 how do they relate in ...,Thank you for your thoughtful question about h...,None,Chapter 2 discusses the minimum weeks of instr...,0,0,1,4.350886,357856f9-6e21-4f0b-893a-43f18d817161,9eeda84d-0ebe-4849-a2f3-e47523dbd329
4,Hwo do academic calenders comply with 34 CFR 6...,Thank you for your thoughtful question. Based ...,None,Academic calenders must define an academic yea...,1,0,1,5.546829,c719285b-5354-4696-b502-cf41c9e94cd9,5e9a4898-544d-4228-83c1-800ab45b7447
5,How does the inclusion of clinical work in sta...,Thank you for your thoughtful question. Based ...,None,The inclusion of clinical work in standard ter...,1,1,1,5.416520,c9a07bf5-0785-44fa-b407-5674f07667c4,591ce424-4b11-493e-99a5-c39b8ba2029e
6,How does completing additional hours or weeks ...,I can see that you’re trying to understand how...,None,The context explains that in clock-hour or non...,1,1,1,8.773632,0190f70a-d6b4-453f-8b4d-919ee56d08c3,f1f98d60-2584-4993-a6a4-226ae176d88c
7,How do scheduled payment periods and installme...,Thank you for your thoughtful question. It sho...,None,Scheduled payment periods determine the timing...,1,0,1,6.432794,dcec7e14-e485-42f8-965a-6847a23b4b36,f7a2402b-9b28-49f1-b1c5-983f36730505
8,What is the significance of Title IV in relati...,Thank you for your thoughtful question about T...,None,The payment period is applicable to all Title ...,1,1,1,5.322762,3776eeb5-9d49-43d0-803e-d835b10eb4c0,94cd8cb2-9297-4eab-bfd0-fd675be79539
9,What is the significance of Volume 8 in the co...,Thank you for your thoughtful question. Based ...,None,Inclusion of clinical work in a standard term ...,1,1,1,2.797664,5805d52c-401c-4096-bf4f-4a26c242022a,7ac60940-0880-48a8-97a6-45aef1219824


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

- First experiment
![First experiment](shiny-opinion-64.png)
- Second experiment
![Second experiment](cold-education-13.png)


The most notable difference between the two experiments is the improvement in the empathy metric. This change can be attributed to the updated prompt used in the second experiment, which specifically instructed the model to provide more empathetic responses. As a result, the answers generated by the second chain were more considerate and supportive, leading to higher empathy scores.
It appears that enhancing the prompt to encourage empathy had an impact on both the helpfulness and correctness metrics. This is likely because the model, guided by the new prompt, prioritized empathetic language, which may have shifted its focus away from strictly factual or concise answers. As a result, the overall answer style became more supportive, but potentially at the expense of precision or directness in addressing the original question.